# Lekiwi

## Calibration

In [ ]:
!poetry run python -m lerobot.calibrate \
    --teleop.type=so100_leader \
    --teleop.port=/dev/tty.usbmodem5A460849101 \ 
    --teleop.id=leader_s101

In [ ]:
from lerobot.common.robots.lekiwi import LeKiwi, LeKiwiConfig

In [ ]:
print("Configuring LeKiwi")
robot_config = LeKiwiConfig()
robot = LeKiwi(robot_config)

print("Connecting LeKiwi")
robot.connect()

In [ ]:
robot.search_for_object('fork')

In [ ]:
arm_action = {'arm_shoulder_pan.pos': -5.0, 'arm_shoulder_lift.pos': -98.92428630533719, 'arm_elbow_flex.pos': 99.27895448400182, 'arm_wrist_flex.pos': 19.973137973137966, 'arm_wrist_roll.pos': -0.31746031746031633, 'arm_gripper.pos': 0.867244829886591}

base_action = {'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -10.0}

robot.send_action(arm_action | base_action)

In [ ]:
robot.stop_base()

In [ ]:
obs = robot.get_observation()
print("Observation:", obs['observation.state'])

In [ ]:
obs = robot.get_observation()

front_image = obs['observation.images.front']
wrist_image = obs['observation.images.wrist']
print(obs['observation.images.front'].shape)

In [ ]:
# front image is an array of shape (480, 640, 3)
# Save the image to a file
import cv2
cv2.imwrite('front_image.png', front_image)   
cv2.imwrite('wrist_image.png', wrist_image)  

In [ ]:
from ultralytics import YOLO
import cv2

# Load YOLOv8 pretrained model (YOLOv8n is the smallest version)
model = YOLO("yolov8n.pt")  # or "yolov8s.pt" for more accuracy

# Load the image from observation
obs = robot.get_observation()
front_image = obs['observation.images.front']

# Save or convert it to match input format
image_bgr = front_image[:, :, ::-1]  # Convert RGB (obs) to BGR (OpenCV)

# Run detection
results = model.predict(image_bgr, conf=0.3)  # You can adjust confidence threshold

# Parse results
fork_detected = False
fork_position = None
output_image = image_bgr.copy()

for r in results:
    for box in r.boxes:
        cls_id = int(box.cls[0].item())
        class_name = model.names[cls_id]

        if class_name.lower() == "fork":
            fork_detected = True
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            fork_position = ((x1 + x2) // 2, (y1 + y2) // 2)

            # Draw bounding box
            cv2.rectangle(output_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(output_image, "Fork", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# Save result
cv2.imwrite("fork_detection_result.png", output_image)

print(f"Fork detected: {fork_detected}")
if fork_detected:
    print(f"Fork center position (in image): {fork_position}")


In [ ]:
from ultralytics import YOLO
# Load YOLOv8 model
# model = YOLO('yolov8n.pt')  # or yolov8s.pt for better accuracy
model = YOLO('yolov8s.pt') 

In [ ]:
SEARCH_ARM_ACTION = {'arm_shoulder_pan.pos': -5.0, 'arm_shoulder_lift.pos': -98.92428630533719, 'arm_elbow_flex.pos': 99.27895448400182, 'arm_wrist_flex.pos': 19.973137973137966, 'arm_wrist_roll.pos': -0.31746031746031633, 'arm_gripper.pos': 0.867244829886591}
SLOW_ROTATE_BASE_ACTION = {'x.vel': 0.0, 'y.vel': 0.0, 'theta.vel': -10.0}


In [ ]:

robot.send_action(SEARCH_ARM_ACTION | SLOW_ROTATE_BASE_ACTION)

# Open video capture from USB camera
cap = robot.cameras['front'].videocapture
print(f"Camera is opened from: {cap.isOpened()}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to read from camera. Stopping the robot.")
        robot.stop_base()
        break

    # Run YOLO detection
    results = model.predict(frame, conf=0.3, verbose=False)
    print("Prediction")

    if any(model.names[box.cls[0].item()] == 'fork' for r in results for box in r.boxes):
        print("Fork detected, stopping the robot.")
        robot.stop_base()
        break

